In [171]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer/customer.csv
/kaggle/input/cars-dataset/cars.csv


In [172]:
df = pd.read_csv("/kaggle/input/customer/customer.csv")

In [173]:
df.head()

,age,gender,review,education,purchased
0,30,Female,Average,School,No
1,68,Female,Poor,UG,No
2,70,Female,Good,PG,No
3,72,Female,Good,PG,No
4,16,Female,Average,UG,No


<p>Catergorical Data is - object type</p>
<p>we convert it into number before training the model</p>

<h3>Approach according to need</h3>
<p>Two types of data - Nominal (no order), Ordinal (with order)</p>
<p>Example of Nominal - Weather (Rainy, Cloudy, Sunny)</p>
<p>Example of Ordinal - Experience (Good > Okish > Bad)</p>

<p>For input ordinal data - OrdinalEncoder(), for output ordinal data - LabelEncoder()</p>
<p>For Nominal input and output data - OneHotEncoding()</p>
<p><b>First lets learn to handle Ordinal Data, and ignore the Nominal Data<b></p>

In [174]:
ordinal_output_col = ['purchased']
ordinal_input_cols = ['review','education'];
ordinal_input_cols

['review', 'education']

In [175]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = LinearRegression()
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [176]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df.iloc[:,2:4],df.iloc[:,-1],test_size=0.2)

In [177]:
X_train.head()

,review,education
28,Poor,School
33,Good,PG
32,Average,UG
49,Good,UG
31,Poor,School


<p>Ordinal Encoding on 'review' ,'education'</p>
<p>Label Encoding on 'purchased'</p>

<h3>Ordinal Encoding</h3>
<p>Notice how we only feed the object ordinal columns</p>

In [178]:
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
oe = OrdinalEncoder(categories=[['Poor','Average','Good'],['School','UG','PG']])
label_X_train[ordinal_input_cols] = oe.fit_transform(X_train[ordinal_input_cols])
label_X_valid[ordinal_input_cols] = oe.transform(X_valid[ordinal_input_cols])

In [179]:
label_X_train.sample(5)

,review,education
10,2.0,1.0
3,2.0,2.0
5,1.0,0.0
34,1.0,0.0
11,2.0,1.0


<h3>Label Encoding</h3>
<p>this converts y_train and y_valid to numpy array</p>

In [180]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder() #cannot decide order here, it will be randomly assigned

# Make copy to avoid changing original data 
label_y_train = y_train.copy()
label_y_valid = y_valid.copy()

# Apply label encoder to output column
label_y_train = le.fit_transform(y_train)
label_y_valid = le.transform(y_valid)

In [181]:
label_y_train

array([0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0])

In [182]:
print("Score after doing Ordinal and Nominal Encoding") 
print(f'{score_dataset(label_X_train, label_X_valid, label_y_train, label_y_valid):.2f}')

Score after doing Ordinal and Nominal Encoding
0.50


<h1>One Hot Encoding</h1>
<p>Now that we are done with Ordinal and Label encoders, we will see OHE with a <b>fresh new dataset</b></p>

In [183]:
data = pd.read_csv("/kaggle/input/cars-dataset/cars.csv");

In [184]:
data.head()

,brand,km_driven,fuel,owner,selling_price
0,Maruti,145500,Diesel,First Owner,450000
1,Skoda,120000,Diesel,Second Owner,370000
2,Honda,140000,Petrol,Third Owner,158000
3,Hyundai,127000,Diesel,First Owner,225000
4,Maruti,120000,Petrol,First Owner,130000


In [185]:
data.nunique()

brand             32
km_driven        921
fuel               4
owner              5
selling_price    677
dtype: int64

<h2>OneHotEncoding using Pandas</h2>
<p>OneHotEncoding can be performed using pandas and sklearn. However, pandas does not retain the order of columns, so it is not commonly used for this purpose</p>

<p>In Machine Learning, it is important to ensure that your columns are not dependent on or have mathematical relationships with each other, which is known as <em>multicollinearity</em>. Hence, your columns should not exhibit multicollinearity.

To address multicollinearity when using OneHotEncoding, we remove one column. If you have 'n' categories in a column, after OneHotEncoding, you will have 'n-1' columns remaining. These resulting columns from OneHotEncoding are referred to as <em>dummy variables</em>. The creation of these dummy variables introduces multicollinearity, which is known as the <em>"dummy variable trap."</em> To avoid falling into this trap, we remove one column.</p>

In [186]:
#we will not use this for brand as of now, we will deal with large categories colums later ahead
pd.get_dummies(data, columns = ['fuel','owner'], drop_first=True)

,brand,km_driven,selling_price,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti,145500,450000,1,0,0,0,0,0,0
1,Skoda,120000,370000,1,0,0,0,1,0,0
2,Honda,140000,158000,0,0,1,0,0,0,1
3,Hyundai,127000,225000,1,0,0,0,0,0,0
4,Maruti,120000,130000,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai,110000,320000,0,0,1,0,0,0,0
8124,Hyundai,119000,135000,1,0,0,1,0,0,0
8125,Maruti,120000,382000,1,0,0,0,0,0,0
8126,Tata,25000,290000,1,0,0,0,0,0,0


<h2>OneHotEncoding using Sklearn</h2>
<p>OneHotEncoding has several parameters. The "sparse" parameter determines whether the output will be a dataframe (if True) or a numpy array (if False). The "drop" parameter, when set to "first," drops the first column. Additionally, the "dtype" parameter, when set to "numpy integer," performs OneHotEncoding with 0 and 1 integers instead of floats.</p>

In [187]:
my_cols = ['fuel','owner']

In [188]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data.iloc[:,0:4],data.iloc[:,-1],test_size=0.2,random_state=2)
X_train.head()

,brand,km_driven,fuel,owner
5571,Hyundai,35000,Diesel,First Owner
2038,Jeep,60000,Diesel,First Owner
2957,Hyundai,25000,Petrol,First Owner
7618,Mahindra,130000,Diesel,Second Owner
6684,Hyundai,155000,Diesel,First Owner


In [189]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first',sparse=False,dtype=np.int32)
X_train_new = ohe.fit_transform(X_train[my_cols]) #use .values if sparse=True
X_test_new = ohe.transform(X_test[my_cols])

#add all featues together
final_X_train = np.hstack((X_train[['brand','km_driven']].values,X_train_new))

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [190]:
X_train_new

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [191]:
final_X_train

array([['Hyundai', 35000, 1, ..., 0, 0, 0],
       ['Jeep', 60000, 1, ..., 0, 0, 0],
       ['Hyundai', 25000, 0, ..., 0, 0, 0],
       ...,
       ['Tata', 15000, 0, ..., 0, 0, 0],
       ['Maruti', 32500, 1, ..., 1, 0, 0],
       ['Isuzu', 121000, 1, ..., 0, 0, 0]], dtype=object)

<h3>now handle columns of high categories</h3>

In [192]:
counts = data['brand'].value_counts()
counts

Maruti           2448
Hyundai          1415
Mahindra          772
Tata              734
Toyota            488
Honda             467
Ford              397
Chevrolet         230
Renault           228
Volkswagen        186
BMW               120
Skoda             105
Nissan             81
Jaguar             71
Volvo              67
Datsun             65
Mercedes-Benz      54
Fiat               47
Audi               40
Lexus              34
Jeep               31
Mitsubishi         14
Force               6
Land                6
Isuzu               5
Kia                 4
Ambassador          4
Daewoo              3
MG                  3
Ashok               1
Opel                1
Peugeot             1
Name: brand, dtype: int64

In [193]:
threshold = 100
data['brand'].nunique()

32

In [194]:
replace_cols = counts[counts <= threshold].index
replace_cols

Index(['Nissan', 'Jaguar', 'Volvo', 'Datsun', 'Mercedes-Benz', 'Fiat', 'Audi',
       'Lexus', 'Jeep', 'Mitsubishi', 'Force', 'Land', 'Isuzu', 'Kia',
       'Ambassador', 'Daewoo', 'MG', 'Ashok', 'Opel', 'Peugeot'],
      dtype='object')

In [195]:
OHE_high_cat_col = pd.get_dummies(data['brand'].replace(replace_cols, 'uncommon'))
OHE_high_cat_col.head()

,BMW,Chevrolet,Ford,Honda,Hyundai,Mahindra,Maruti,Renault,Skoda,Tata,Toyota,Volkswagen,uncommon
0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0
